In [1]:
import datetime
def getDate(timestamp):
    date = datetime.datetime.fromtimestamp(int(timestamp) / 1000.0)
    date = date.strftime('%Y-%m-%d %H:%M:%S')
    return date

def getTime(date):
    A = date.split()[1]
    A = A.split(":")[0]
    return A

In [2]:
import json
import re
tweets = json.load(open("gg2013.json"))

for tweet in tweets:
    text = tweet['text']
    timestamp = tweet['timestamp_ms']
    date = getDate(timestamp)
    time = getTime(date)
    if int(time)>=20:    
        if re.search('best performance by an actress',text):
            print(time)
            print(tweet)

20
{'text': 'Lena Dunham wins best performance by an actress in a TV musical or comedy for #Girls http://t.co/VCUv6LqO #GoldenGlobes', 'user': {'screen_name': 'ABCNewsEnt', 'id': 83866930}, 'id': 290652831843749889, 'timestamp_ms': 1358132009000}
20
{'text': 'RT @ABCNewsEnt: Lena Dunham wins best performance by an actress in a TV musical or comedy for #Girls http://t.co/VCUv6LqO #GoldenGlobes', 'user': {'screen_name': 'MeelaPeela', 'id': 61225801}, 'id': 290653755718918144, 'timestamp_ms': 1358132230000}
21
{'text': 'RT @ABCNewsEnt: Lena Dunham wins best performance by an actress in a TV musical or comedy for #Girls http://t.co/VCUv6LqO #GoldenGlobes', 'user': {'screen_name': 'jetrui_dsw', 'id': 990989762}, 'id': 290654945760378880, 'timestamp_ms': 1358132513000}


In [10]:
people_award = {}

film_award = {}

film_award['best screenplay - motion picture'] = (r'[Bb]est [Ss]creenplay')

people_award['best director - motion picture'] = (r'[Bb]est [Dd]irector')

people_award['best performance by an actress in a television series - comedy or musical'] = (r'[Bb]est [Pp]erformance .* [Aa]ctress .* [Tt][Vv].* [Mm]usical')

film_award['best foreign language film'] = (r'[Ff]oreign [Ll]anguage [Ff]ilm')


people_award['best performance by an actor in a supporting role in a motion picture'] = (r'([Aa]ct[oe]r.*[Ss]upporting|[Ss]upporting [Aa]ct[oe]r) (in a|in) [Mm]otion')

people_award['best performance by an actress in a supporting role in a series, mini-series or motion picture made for television'] = (r'([Ss]upporting.*[Aa]ctress.*[Ss]eries|[Aa]ctress.*[Ss]upporting [Rr]ole.*[Ss]eries)')

film_award['best motion picture - comedy or musical'] = (r'[Bb]est [Mm]otion [Pp]icture .*[Co]medy .* [Mm]usical')

people_award['best performance by an actress in a motion picture - comedy or musical'] = (r'[Bb]est [Pp]erformance .* [A]ctress [Ii]n [Aa] [Mm]otion [Pp]icture.*[Cc]omedy')

film_award['best mini-series or motion picture made for television'] = (r'[Bb]est [Mm]ini.*[Ss]eries .*[Mm]otion [Pp]icture.*([Tt][Vv]|[Tt]elevision)')

film_award['best original score - motion picture '] = (r'[Bb]est [Oo]riginal [Ss]core.*[Mm]otion [Pp]icture')

people_award['best performance by an actress in a television series - drama'] = (r'[Bb]est [Aa]ctress( in a| in)?,? ([Tt][Vv]|[Tt]elevision).*[Dd]rama')

people_award['# best performance by an actress in a motion picture - drama'] = (r'[Bb]est [Aa]ctress.*[Mm]otion [Pp]icture.*[Dd]rama')

people_award['cecil b. demille award'] = (r'[Cc]ecil.*[Dd]emille')

people_award['best performance by an actor in a motion picture - comedy or musical'] = (r'[Bb]est [Pp]erformance .* [Aa]ct[oe]r.* [Mm]otion [Pp]icture.*[Cc]omedy')

people_award['best performance by an actor in a supporting role in a series, mini-series or motion picture made for television'] = (r'[Ss]upporting.*[Aa]ct[oe]r.*[Mm]ini')

people_award[' best performance by an actress in a supporting role in a motion picture'] = (r'([Bb]est|[Ll]eading) ([Ss]upporting [Aa]ctress|[Aa]ctress.*[Ss]upporting).*([Mm]otion [Pp]icture|[Ff]ilm)')

film_award[' best motion picture - drama'] = (r'([Bb]est|[Ll]eading) ((([Mm]otion)?\s?[Pp]icture|[Ff]ilm).*[Dd]rama|[Dd]rama.*(([Mm]otion)? [Pp]icture|[Ff]ilm))')

film_award['best television series - drama'] = (r'([Bb]est|[Ll]eading) (([Tt][Vv]|[Ss]eries).*[Dd]rama|([Tt][Vv]|[Ss]eries)|[Dd]rama [Ss]eries)')

people_award[' best performance by an actor in a mini-series or motion picture made for television'] = (r'([Bb]est|[Ll]eading) [Aa]ctor.*[Mm]ini')

people_award['best performance by an actress in a mini-series or motion picture made for television'] = (r'([Bb]est|[Ll]eading) [Aa]ctress.*[Mm]ini')

film_award[' best animated feature film'] = (r'[Bb]est [Aa]nimated')

film_award['best original song - motion picture'] = (r'[Bb]est ([Oo]riginal )?[Ss]ong')

people_award[' best performance by an actor in a motion picture - drama '] = (r'[Bb]est [Aa]ct[oe]r.*(([Mm]otion)?\s?[Pp]icture|[Ff]ilm).*([Dd]rama)?')

film_award[' best television series - comedy or musical'] = (r'[Bb]est ([Tt][Vv]|[Tt]elevision).*[Cc]omedy')

people_award['best performance by an actor in a television series - drama'] = (r'[Bb]est [Aa]ct[oe]r( in a| in)?,? ([Tt][Vv]|[Tt]elevision).*[Dd]rama')

people_award['best performance by an actor in a television series - comedy or musical'] = (r'[Bb]est [Aa]ct[oe]r( in a| in)?,? ([Tt][Vv]|[Tt]elevision).*[Cc]omedy')

In [109]:
import nltk;

import spacy
nlp = spacy.load('en') 

In [128]:
def get_winner(award_name):
    
    if award_name in people_award.keys():
        search_term = people_award.get(award_name)
    else:
        search_term = film_award.get(award_name)
    
    relevant_tweets = []
    for t in tweets:
        text = t['text']
        if re.search(search_term, text) and 'RT' not in text:
            relevant_tweets.append(text)  
            
    possible_answer = []
            
    for t in relevant_tweets:
        tokenized_t = t.lower().split()
        if 'wins' in tokenized_t:
            index = tokenized_t.index("wins")
            before_win = []
            string = tokenized_t[0]
            for i in range(1, index): 
                string += ' ' + tokenized_t[i]
            before_win.append(string)
            
#             print(before_win)
            
            sents_to_extract = []
            
            for s in before_win:
            
#             sents_to_extract = nlp(s) 
            
                names = [ee for ee in nlp(s).ents if ee.label_ == 'PERSON']
                possible_answer.append(names)
                
            
#     name_pair = []
#     for possible_name_pair in possible_answer:
#         if nltk.pos_tag(possible_name_pair)[0] == 'NNP' and nltk.pos_tag(possible_name_pair)[1] == 'NNP':
#             name_pair.append(possible_name_pair)
        
    
    print(possible_answer)
    #return relevant_tweets

In [127]:
get_winner('best performance by an actress in a mini-series or motion picture made for television')

[[julianne moore], [julianna moore], [julianne moore], [julianne moore], [julianne moore], [julianne moore], [julianne moore], [julianne moore], [julianne moore], [julianne moore], [[perez, julianne moore], [julianne moore], [goldenglobes claire], [julianne moore]]


In [ ]:
import spacy
en = spacy.load('en')

sents = en(open('input.txt').read())
people = [ee for ee in sents.ents if ee.label_ == 'PERSON']